In [1]:
import sklearn
from sklearn.datasets import make_circles
# Make 100 Samples
n_samples = 10000
X,y = make_circles(n_samples,noise=0.0625,random_state=42)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
X[:5],y[:5]

(array([[ 0.09898901,  0.85162427],
        [ 1.0948453 , -0.48384393],
        [-0.59993055,  0.76388002],
        [ 1.02473745, -0.33234668],
        [ 0.83608956, -0.52662827]]),
 array([1, 0, 0, 0, 0]))

In [4]:
# Make a DataFrame of circle data
import pandas as pd

In [5]:
circles = pd.DataFrame({"X1":X[:,0],"X2":X[:,1],"y":y})

In [6]:
# Visualizing
plt.scatter(x=X[:,0],y=X[:,1],c=y,cmap=plt.cm.RdYlBu)

In [7]:
X_sample = X[0]
y_sample = y[0]
X_sample,y_sample,X_sample.shape,y_sample.shape

(array([0.09898901, 0.85162427]), 1, (2,), ())

In [8]:
# Turn data into tensors
import torch
torch.__version__

'1.9.1'

In [9]:
X.dtype

dtype('float64')

In [10]:
X = torch.from_numpy(X).type(torch.float32).to(device)
y = torch.from_numpy(y).type(torch.float32).to(device)

In [11]:
X[:2],y[:2]

(tensor([[ 0.0990,  0.8516],
         [ 1.0948, -0.4838]], device='cuda:0'),
 tensor([1., 0.], device='cuda:0'))

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [14]:
len(X_train),len(y_test)

(7500, 2500)

In [15]:
import torch
from torch import nn

# Make device agnositic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
device

'cuda'

In [17]:
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(2,2048) # 
        self.layer_2 = nn.Linear(2048,1)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        # return self.layer_2(self.relu(self.layer_1(X))) # x -> layer_1 -> layer_2
        return self.layer_2(self.layer_1(X))

In [18]:
model_0 = CircleModelV0().to(device)

In [19]:
model_0

CircleModelV0(
  (layer_1): Linear(in_features=2, out_features=2048, bias=True)
  (layer_2): Linear(in_features=2048, out_features=1, bias=True)
  (relu): ReLU()
)

In [20]:
list(model_0.parameters())

[Parameter containing:
 tensor([[-0.1729, -0.0314],
         [ 0.5891, -0.4045],
         [ 0.0348, -0.4718],
         ...,
         [ 0.2985, -0.6466],
         [-0.5384,  0.1685],
         [-0.6692, -0.0179]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([ 0.4969, -0.2854, -0.3357,  ...,  0.2665,  0.2368, -0.3879],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[-0.0107,  0.0089,  0.0019,  ...,  0.0035,  0.0048,  0.0134]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([-0.0199], device='cuda:0', requires_grad=True)]

In [21]:
# model_0 = nn.Sequential(
#     nn.Linear(in_features=2,out_features=64),
#     nn.Linear(64,1)
# ).to(device)

In [22]:
untrained_preds = model_0(X_test)

In [23]:
untrained_preds[0],y_test[0]

(tensor([-0.0693], device='cuda:0', grad_fn=<SelectBackward>),
 tensor(1., device='cuda:0'))

In [24]:
loss_fn = nn.BCEWithLogitsLoss() # has the sigmoid function builtin
# BCELoss() requries sigmoid to be builtin to the model itself

In [25]:
optimizer = torch.optim.Adam(model_0.parameters())

In [26]:
epochs = 10
batch_size = 32

In [27]:
# Calculate accuracy
def accuracy_fn(y_true,y_preds):
    correct = torch.eq(y_true,y_preds).sum().item() # gives a False True list -> Tensor no. of true > just normal no.
    acc = correct/len(y_preds)*100
    return acc

In [28]:
y_logits = model_0(X_test)

In [29]:
y_preds_probs = torch.sigmoid(y_logits)

In [30]:
y_preds_probs.round()

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], device='cuda:0', grad_fn=<RoundBackward>)

In [31]:
y_pred_labels = torch.round(torch.sigmoid(model_0(X_test)))

In [32]:
y_preds = torch.round(y_preds_probs)

In [33]:
y_preds.squeeze()

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)

In [34]:
test_loss_iter = []
train_loss_iter = []
train_accuracy_iter = []
test_accuracy_iter = []

In [35]:
from tqdm import tqdm

In [36]:
# %%time
# epochs = 100
# batch_size = 32

# for epoch in tqdm(range(epochs)):
#     for i in range(0,len(X_train),batch_size):
#         X_batch = X_train[i:i+batch_size]
#         y_batch = y_train[i:i+batch_size]
#         preds = model_0(X_batch)
#         true_preds = torch.round(torch.sigmoid(preds.squeeze()))
#         loss = loss_fn(preds.squeeze(),y_batch.squeeze())
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     with torch.inference_mode():
#         y_test_preds = model_0(X_test)
#         loss_test = loss_fn(y_test_preds.squeeze(),y_test.squeeze())
#         true_test_preds = torch.round(torch.sigmoid(y_test_preds))
#     train_loss_iter.append(loss.cpu().detach().numpy())
#     test_loss_iter.append(loss_test.cpu().detach().numpy())
#     train_accuracy_iter.append(accuracy_fn(y_batch,true_preds))
#     test_accuracy_iter.append(accuracy_fn(y_test,true_test_preds))

In [37]:
for epoch in tqdm(range(epochs)):
    model_0.train()
    y_logists = model_0(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))
    loss = loss_fn(y_logists,y_train)
    acc = accuracy_fn(y_true=y_train,y_preds=y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model_0.eval()
    with torch.inference_mode():
        test_logits = model_0(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        
        test_loss = loss_fn(test_logits,y_test)
        test_acc = accuracy_fn(y_true=y_test,y_preds=test_pred)
        
print(f"""
        Loss : {loss}
        Accuracy : {acc}
        Test Loss : {test_loss}
        Test Accuracy : {test_acc}
        """)

In [38]:
import requests
from pathlib import Path

# Download helper functions from PyTorch repo
if not Path("helper_functions.py").is_file():
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
    with open("helper_functions.py","wb") as f:
        f.write(request.content)

In [39]:
from helper_functions import *

In [40]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("Train")
plot_decision_boundary(model_0,X_train,y_train)
plt.subplot(1,2,2)
plt.title("Test")
plot_decision_boundary(model_0,X_test,y_test)

In [41]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = nn.ReLU()
        self.linear1 = nn.Linear(2,256)
        self.linear2 = nn.Linear(256,512)
        self.linear3 = nn.Linear(512,1024)
        self.linear4 = nn.Linear(1024,512)
        self.linear5_output = nn.Linear(512,1)
    
    def forward(self,X):
        X = self.activation(self.linear1(X))
        X = self.activation(self.linear2(X))
        X = self.activation(self.linear3(X))
        X = self.activation(self.linear4(X))
        X = self.linear5_output(X)
        return X

In [42]:
model = ClassificationModel().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [43]:
epochs = 250
batch_size = 32

In [44]:
import wandb

In [45]:
wandb.init(project="02",name="Adjusted")
for epoch in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        model.train()
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch).squeeze()
        norm_preds = torch.round(torch.sigmoid(preds))
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
        with torch.inference_mode():
            train_preds = model(X_train).squeeze()
            test_preds = model(X_test).squeeze()
            loss_test = criterion(test_preds,y_test)
            loss_train = criterion(train_preds,y_train)
            train_preds = torch.round(torch.sigmoid(train_preds))
            test_preds = torch.round(torch.sigmoid(test_preds))
            acc_train = accuracy_fn(y_train,train_preds)
            acc_test = accuracy_fn(y_test,test_preds)
            wandb.log({
                "Train Loss":loss_train,
                "Test Loss":loss_test,
                "Train Accuracy": acc_train,
                "Test Accuracy": acc_test
            })
wandb.finish()